In [1]:
import os
import pandas as pd
import json

os.chdir("d:/functional-prediction")

In [2]:
df_path = pd.read_csv("processed/pathway_relation_review.csv", dtype=str).fillna("")

df_from = df_path[["from_entity", "from_type"]]
df_from[["entity", "type"]] = df_from[["from_entity", "from_type"]]
df_from = df_from[["entity", "type"]]

df_to = df_path[["to_entity", "to_type"]]
df_to[["entity", "type"]] = df_to[["to_entity", "to_type"]]
df_to = df_to[["entity", "type"]]

df_ent = pd.concat([df_from, df_to], axis=0).drop_duplicates()

df_ent.groupby(["entity"])["type"].count().reset_index(name="count").sort_values(by=["count"], ascending=False)[:50]

d:\anaconda3\envs\drkg\lib\site-packages\pandas\core\frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,entity,count
0,(-)-carbovir 5'-diphosphate,1
1153,desmethyl clomipramine,1
1164,desmethylsertraline,1
1163,desmethylnortriptyline,1
1162,desmethylnaproxen,1
1161,desmethylimatinib,1
1160,desmethylerythromycin,1
1159,desmethyldoxepin,1
1158,desmethylclozapine glucuronide,1
1157,desmethylbosutinib,1


In [3]:
for key, content in df_ent.groupby(["type"]):
    column = "{k}_name:ID({k})".format(k=key)
    content[[column]] = content[["entity"]]
    content = content[[column]]
    content[":LABEL"] = [key] * len(content)
    content[["display"]] = content[[column]]
    content.to_csv("processed/{}_entity.csv".format(key), index=False)

d:\anaconda3\envs\drkg\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [4]:
for key, content in df_path.groupby(["from_type", "to_type"]):
    start_column = ":START_ID({})".format(key[0])
    end_column = ":END_ID({})".format(key[1])
    content[[start_column]] = content[["from_entity"]]
    content[[end_column]] = content[["to_entity"]]
    content[":TYPE"] = "pathway"
    for col in ["from_entity", "from_type", "to_entity", "to_type"]:
        content.pop(col)
        
    content.to_csv("processed/{}_{}_rel.csv".format(key[0], key[1]), index=False)

In [35]:
# generate neo4j import script
entity_list = []
rel_list = []

for fn in os.listdir("processed"):
    if "_rel.csv" in fn:
        rel_list.append(fn)
    
    if "_entity.csv" in fn:
        entity_list.append(fn)
        
import_str = "neo4j-admin import --database pathway ^\n"
entity_list = ["--nodes=csv_upload/pathway/{} ^".format(x) for x in entity_list]
rel_list = ["--relationships=csv_upload/pathway/{} ^".format(x) for x in rel_list]
import_str += "\n".join(entity_list) + "\n"
import_str += "\n".join(rel_list)

print(import_str)

neo4j-admin import --database pathway ^
--nodes=csv_upload/pathway/compound_entity.csv ^
--nodes=csv_upload/pathway/disease_entity.csv ^
--nodes=csv_upload/pathway/drug_entity.csv ^
--nodes=csv_upload/pathway/function_entity.csv ^
--nodes=csv_upload/pathway/gene_entity.csv ^
--nodes=csv_upload/pathway/metabolizer_entity.csv ^
--relationships=csv_upload/pathway/compound_compound_rel.csv ^
--relationships=csv_upload/pathway/compound_disease_rel.csv ^
--relationships=csv_upload/pathway/compound_drug_rel.csv ^
--relationships=csv_upload/pathway/compound_gene_rel.csv ^
--relationships=csv_upload/pathway/disease_disease_rel.csv ^
--relationships=csv_upload/pathway/disease_gene_rel.csv ^
--relationships=csv_upload/pathway/drug_compound_rel.csv ^
--relationships=csv_upload/pathway/drug_disease_rel.csv ^
--relationships=csv_upload/pathway/drug_drug_rel.csv ^
--relationships=csv_upload/pathway/drug_gene_rel.csv ^
--relationships=csv_upload/pathway/gene_compound_rel.csv ^
--relationships=csv_uplo

#### neo4j dump
```
neo4j-admin dump --database=pathway --to=d:/neo4j_dump/pathway.db.<date>.dump
```

In [28]:
os.chdir("d:/pgkb_graph")
df_guide = pd.read_csv("static/pgkb_guideline.csv", dtype=str).fillna("")
gene_list = sorted(set(df_guide["gene"].values) - {"HLA_B"})
df_guide = df_guide[df_guide["gene"].isin(gene_list)]

In [7]:
drug_list = sorted(set(df_guide["drug"].values))

In [26]:
os.chdir("d:/functional-prediction")

In [9]:
set(df_guide["metabolizer"].values)

{'Extensive Metabolizer',
 'Intermediate Metabolizer',
 'Low Function',
 'Normal Metabolizer',
 'Poor Metabolizer',
 'Rapid Metabolizer',
 'Ultrarapid Metabolizer'}

In [10]:
for index, row in df_guide.iterrows():
    if row["metabolizer"] == "Extensive Metabolizer":
        row["metabolizer"] = 'Normal Metabolizer'
    if row["metabolizer"] == "Low Function":
        row["metabolizer"] = "Poor Metabolizer"
        
    row["metabolizer"] = "{} {}".format(row["gene"], row["metabolizer"])

In [11]:
# 处理 gene - metabolizer 关系
df_gm = df_guide[["gene", "metabolizer"]].drop_duplicates()
df_gm[[":START_ID(gene)", ":END_ID(metabolizer)"]] = df_gm[["gene", "metabolizer"]]
df_gm = df_gm[[":START_ID(gene)", ":END_ID(metabolizer)"]]
df_gm[":TYPE"] = ["gene_metabolizer_relation"] * len(df_gm)
df_gm.to_csv("processed/gene_metabolizer_rel.csv", index=False)

In [12]:
# 处理 gene 实体
df_gene = df_gm[[":START_ID(gene)"]]
df_gene[["gene_name:ID(gene)"]] = df_gene[[":START_ID(gene)"]]
df_gene[["display"]] = df_gene[["gene_name:ID(gene)"]]
df_gene[[":LABEL"]] = ["gene"] * len(df_gene)
df_gene = df_gene[["gene_name:ID(gene)", ":LABEL", "display"]]

df_gene1 = pd.read_csv("processed/gene_entity.csv", dtype=str).fillna("")

df_gene = pd.concat([df_gene, df_gene1], axis=0).drop_duplicates()

df_gene.to_csv("processed/gene_entity.csv", index=False)

d:\anaconda3\envs\drkg\lib\site-packages\pandas\core\frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [13]:
# 处理 drug 实体
df_drug = df_guide[["drug"]].drop_duplicates()
df_drug[["drug_name:ID(drug)"]] = df_drug[["drug"]]  
df_drug[["display"]] = df_drug[["drug"]]
df_drug[[":LABEL"]] = ["drug"] * len(df_drug)
df_drug = df_drug[["drug_name:ID(drug)", ":LABEL", "display"]]
df_drug1 = pd.read_csv("processed/drug_entity.csv", dtype=str).fillna("")
df_drug = pd.concat([df_drug, df_drug1], axis=0).drop_duplicates()
df_drug.to_csv("processed/drug_entity.csv", index=False)

In [14]:
df_meta = df_guide[["metabolizer"]].drop_duplicates()
df_meta[["metabolizer_name:ID(metabolizer)"]] = df_meta[["metabolizer"]]
df_meta[["display"]] = df_meta[["metabolizer"]]
df_meta[[":LABEL"]] = ["metabolizer"] * len(df_meta)
df_meta = df_meta[["metabolizer_name:ID(metabolizer)", ":LABEL", "display"]]

In [15]:
# 处理 metabolizer - drug 关系
df_guide[[":START_ID(metabolizer)", ":END_ID(drug)"]] = df_guide[["metabolizer", "drug"]]
df_guide.pop("metabolizer")
df_guide.pop("drug")
df_guide[":TYPE"] = ["guideline"] * len(df_guide)
df_guide.to_csv("processed/metabolizer_drug_rel.csv", index=False)

In [37]:
# 处理 (gene)-[cpic_guideline]-(drug) 关系
os.chdir("d:/pgkb_graph")
df_cpic = pd.read_csv("static/cpic_evidence.csv", dtype=str).fillna("")
df_cpic = df_cpic[(df_cpic["drug"] != "") & (df_cpic["gene"] != "")]

# drug节点融合
df_drug = df_cpic[["drug"]].drop_duplicates()
df_drug[["drug_name:ID(drug)"]] = df_drug[["drug"]]  
df_drug[["display"]] = df_drug[["drug_name:ID(drug)"]]
df_drug[[":LABEL"]] = ["drug"] * len(df_drug)
df_drug = df_drug[["drug_name:ID(drug)", ":LABEL", "display"]]

os.chdir("d:/functional-prediction")
df_drug1 = pd.read_csv("processed/drug_entity.csv", dtype=str).fillna("")
df_drug = pd.concat([df_drug, df_drug1], axis=0).drop_duplicates()
df_drug.to_csv("processed/drug_entity.csv", index=False)

# gene节点融合
df_gene = df_cpic[["gene"]].drop_duplicates()
df_gene[["gene_name:ID(gene)"]] = df_gene[["gene"]]
df_gene[["display"]] = df_gene[["gene_name:ID(gene)"]]
df_gene[[":LABEL"]] = ["gene"] * len(df_gene)
df_gene = df_gene[["gene_name:ID(gene)", ":LABEL", "display"]]
df_gene1 = pd.read_csv("processed/gene_entity.csv", dtype=str).fillna("")
df_gene = pd.concat([df_gene, df_gene1], axis=0).drop_duplicates()
df_gene.to_csv("processed/gene_entity.csv", index=False)

df_cpic[[":START_ID(gene)", ":END_ID(drug)"]] = df_cpic[["gene", "drug"]]
df_cpic.pop("gene")
df_cpic.pop("drug")
df_cpic[":TYPE"] = ["cpic_guideline"] * len(df_cpic)
df_cpic.to_csv("processed/gene_drug_cpic_rel.csv", index=False)


In [38]:
# 处理 (gene)-[drug_label]-(drug) 关系
# 处理 (gene)-[cpic_guideline]-(drug) 关系
os.chdir("d:/pgkb_graph")
df_label = pd.read_csv("static/drug_gene_label.csv", dtype=str).fillna("")
df_label = df_label[(df_label["drug"] != "") & (df_label["variant"] != "")]

# drug节点融合
df_drug = df_label[["drug"]].drop_duplicates()
df_drug[["drug_name:ID(drug)"]] = df_drug[["drug"]]  
df_drug[["display"]] = df_drug[["drug_name:ID(drug)"]]
df_drug[[":LABEL"]] = ["drug"] * len(df_drug)
df_drug = df_drug[["drug_name:ID(drug)", ":LABEL", "display"]]

os.chdir("d:/functional-prediction")
df_drug1 = pd.read_csv("processed/drug_entity.csv", dtype=str).fillna("")
df_drug = pd.concat([df_drug, df_drug1], axis=0).drop_duplicates()
df_drug.to_csv("processed/drug_entity.csv", index=False)

# gene节点融合
df_gene = df_label[["variant"]].drop_duplicates()
df_gene[["gene_name:ID(gene)"]] = df_gene[["variant"]]
df_gene[["display"]] = df_gene[["gene_name:ID(gene)"]]
df_gene[[":LABEL"]] = ["gene"] * len(df_gene)
df_gene = df_gene[["gene_name:ID(gene)", ":LABEL", "display"]]
df_gene1 = pd.read_csv("processed/gene_entity.csv", dtype=str).fillna("")
df_gene = pd.concat([df_gene, df_gene1], axis=0).drop_duplicates()
df_gene.to_csv("processed/gene_entity.csv", index=False)

df_label[[":START_ID(gene)", ":END_ID(drug)"]] = df_label[["variant", "drug"]]
df_label.pop("variant")
df_label.pop("drug")
df_label[":TYPE"] = ["drug_label"] * len(df_label)
df_label.to_csv("processed/gene_drug_pgkb_rel.csv", index=False)


In [17]:
os.chdir("d:/functional-prediction")
df_meta_match = pd.read_csv("data/metabolizer_function_review.csv", dtype=str).fillna("")
df_meta_match = df_meta_match[df_meta_match["json_str"] != ""]

In [18]:
json.loads(df_meta_match.iloc[1]["json_str"])

[['no', 'no'], ['decreased', 'decreased'], ['no', 'decreased']]

In [19]:
match_dict = {
    "metabolizer": [],
    "function_pair": []
}
for index, row in df_meta_match.iterrows():
    metabolizer = "{} {}".format(row["gene"], row["metabolizer"])
    function_list = json.loads(row["json_str"])
    for fl in function_list:
        fl = " / ".join(sorted(fl))
        match_dict["function_pair"].append(fl)
        match_dict["metabolizer"].append(metabolizer)

In [20]:
df_match = pd.DataFrame(match_dict)

In [21]:
# 处理 function 实体
df_fp = df_match[["function_pair"]].drop_duplicates()
df_fp[["function_name:ID(function)"]] = df_fp[["function_pair"]]
df_fp[["display"]] = df_fp[["function_pair"]]
df_fp[[":LABEL"]] = ["function"] * len(df_fp)
df_fp = df_fp[["function_name:ID(function)", "display", ":LABEL"]]

In [22]:
df_fp.to_csv("processed/function_entity.csv", index=False)

In [23]:
# 处理 metabolizer - function 关系
df_match[[":START_ID(metabolizer)", ":END_ID(function)"]] = df_match[["metabolizer", "function_pair"]]
df_match.pop("metabolizer")
df_match.pop("function_pair")
df_match[":TYPE"] = ["function_match"] * len(df_match)
df_match.to_csv("processed/metabolizer_function_rel.csv", index=False)

In [24]:
# 处理 metabolizer 实体
df_new_meta = df_match[[":START_ID(metabolizer)"]]
df_new_meta[["metabolizer_name:ID(metabolizer)"]] = df_new_meta[[":START_ID(metabolizer)"]]
df_new_meta.pop(":START_ID(metabolizer)")

df_new_meta[["display"]] = df_new_meta[["metabolizer_name:ID(metabolizer)"]]
df_new_meta[[":LABEL"]] = ["metabolizer"] * len(df_new_meta)

df_meta = pd.concat([df_new_meta, df_meta]).drop_duplicates()
df_meta.to_csv("processed/metabolizer_entity.csv", index=False)